In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from agents.matchmanager import buildMatchManager
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE
import random


In [ ]:
from agents.matchmanager import MatchManager

from itertools import product

from agents import GreedyAgent, AlphaBetaAgent, RandomAgent
from scenarios import scenarioJunction, scenarioJunctionExo, scenarioTest1v1, scenarioTest2v2
from core.const import RED, BLUE
from agents.ml.simple import SimpleMLAgent
import json
import pandas as pd

In [ ]:
board, state = scenarioJunction()
playerRed = SimpleMLAgent(RED, {'scenario': board.name, 'model': 'RandomForestClassifier'})
playerBlue = SimpleMLAgent(BLUE, {'scenario': board.name, 'model': 'RandomForestClassifier'})


In [ ]:
mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=43)
while not mm.end:
    mm.nextStep()

In [3]:
players=['RandomAgent']
#scenarios=['scenarioTest1v1', 'scenarioTest2v2', 'scenarioTest3v1','scenarioJunction', 'scenarioJunctionExo', 'scenarioRoadblock', 'scenarioBridgeHead']
scenarios=['scenarioJunction']
if __name__ == '__main__':
        for s in scenarios:
            vectors=[]
            winners=[]
            df=pd.DataFrame()
            for i in range(1):

                for p in players:
                    for p1 in players:
                        mm = buildMatchManager('', s, p, p1, seed=random.randint(0,10000))
                        #print('Player 1',p)
                        #print('Player 2',p1)
                        #print('SCenario',s)

                        while not mm.end:
                            mm.nextStep()
                            vectors.append(mm.state.vector())
                        old=df.shape[0]
                        df=pd.DataFrame(vectors)
                        new=df.shape[0]
                        for i in range(new-old):
                            winners.append(mm.winner)

            df.columns=mm.state.vectorInfo()
            df['winner']=winners

            #print('ehi'+s)
            df.to_csv(r'../docs/csv/csv'+s+'.csv')
            #print(df)


In [5]:
mm.states_history

[0:
 {'red': [rTank2(7, 28), rInf21(7, 28), rInf22(6, 29)], 'blue': [bAPC1(37, 26), bInf1(28, 33), bInf2(37, 20)]}
 {'red': {(7, -31, 24: [0, 1], (6, -32, 26: [2]}, 'blue': {(37, -44, 7: [0], (28, -47, 19: [1], (37, -38, 1: [2]}},
 0:
 {'red': [rTank2(7, 28), rInf21(7, 28), rInf22(6, 29)], 'blue': [bAPC1(37, 26), bInf1(28, 33), bInf2(37, 20)]}
 {'red': {(7, -31, 24: [0, 1], (6, -32, 26: [2]}, 'blue': {(37, -44, 7: [0], (28, -47, 19: [1], (37, -38, 1: [2]}},
 0:
 {'red': [rTank2(7, 28), rInf21(7, 28), rInf22(6, 29)], 'blue': [bAPC1(37, 26), bInf1(27, 34), bInf2(37, 20)]}
 {'red': {(7, -31, 24: [0, 1], (6, -32, 26: [2]}, 'blue': {(37, -44, 7: [0], (37, -38, 1: [2], (27, -47, 20: [1]}},
 0:
 {'red': [rTank2(7, 28), rInf21(7, 28), rInf22(6, 29)], 'blue': [bAPC1(37, 26), bInf1(27, 34), bInf2(37, 20)]}
 {'red': {(7, -31, 24: [0, 1], (6, -32, 26: [2]}, 'blue': {(37, -44, 7: [0], (37, -38, 1: [2], (27, -47, 20: [1]}},
 0:
 {'red': [rTank2(7, 28), rInf21(5, 27), rInf22(6, 29)], 'blue': [bAPC1(3

In [ ]:
df

In [ ]:
mm.state.lastAction.__class__.__name__

In [ ]:
mm.state.lastAction.team

In [ ]:
for c in df.columns:
    print(c)

In [ ]:
df.sample(1)

In [ ]:
df['positionZ_red_2_0']

In [ ]:
from core import GM
from core.actions import Action
from core.game.board import GameBoard
from core.game.state import GameState
from agents import Agent, GreedyAgent
import numpy as np
from core.const import RED, BLUE
import pandas as pd
import os.path as op

import joblib

In [ ]:
df

In [ ]:
df['color_red_2_0']

In [ ]:
model=joblib.load('C:\\Users\\Nicol\\Documents\\Master\\SecondoProgetto\\newtechnowar-webui\\code\\models\\Junction_RandomForestClassifier.joblib')

In [ ]:
scores=[]
for figure in state.getFiguresCanBeActivated(RED):
    actions = [GM.actionPassFigure(figure)] + \
              GM.buildAttacks(board, state, figure) + \
              GM.buildMovements(board, state, figure)
    for action in actions:
        newState, outcome = GM.activate(board, state, action)

        X = np.array(newState.vector()).reshape(1, -1)
        cols = newState.vectorInfo()
        df = pd.DataFrame(data=X, columns=cols)
        score = model.predict_proba(df)

        scores.append((score, action))

In [ ]:
scores

In [ ]:
def bestActionRandom(scores):
    sorted_multi_list = sorted(scores, key=lambda x: x[0][0][0])
    if (BLUE):
        bestAction=random.choice(sorted_multi_list[-10:])[1]
    else:
        bestAction=random.choice(sorted_multi_list[:10])[1]
    return bestAction
        
    

In [ ]:
bestScore, bestAction = 0.0, None
#print(sorted(scores, reverse=True))

for probs, action in scores:
    print(probs)
    if (BLUE):
        score = probs[0, 0]
        print("ehi",score)
        print(action)
    else:
        score = probs[0, 1]
        print(score)
    if score >= bestScore:
        bestScore, bestAction = score, action

In [ ]:
from scipy.stats import entropy

In [ ]:
len(scores)

In [ ]:
for probs, action in scores:
    print(entropy(probs[0], base=len(scores)))
